# Geospatial Big Data Analytics Lab 4

Evaggelos Tsogkas 03400120

In this project, we will create a map using folium in order to show the average precipitation according to temperature by month for every city, like the last chart we showed for Athens in Lab 2. 

In [85]:
import numpy as np
import pandas as pd
import geopandas as gpd
import psycopg2
import folium
from folium import plugins
from altair import Chart
import vincent

## Get data from database

First, we query the database to get all the data we need.

In [86]:
# connect to the database
connection = psycopg2.connect(database='greece_weather', user='user', password='user', host='localhost')
connection.autocommit = True
cursor = connection.cursor()

# get data from the database
cities = gpd.GeoDataFrame.from_postgis('SELECT * FROM cities', connection, geom_col = 'coordinates')
weather = pd.io.sql.read_sql('SELECT * FROM weather', connection)
connection.close()

# add a month column to the weather table
weather['month'] = pd.to_datetime(weather['year']*1000 + weather['doy'], format='%Y%j').dt.month

## Folium Map

Next, we create a folium map. We calculate the average precipitation and temperature by month for every city, we add the charts to markers as popups and further style the map with a mini-map, showing coordinates of mouse position and adding various tile options.

In [87]:
# folium map
m = folium.Map(location=[39, 23], zoom_start=7, tiles=None)

marker_cluster = plugins.MarkerCluster()
for i in cities.index:  
    # calculate average precipitation and temperature by month
    city_weather = weather[weather['city_id'] == cities.iloc[i]['id']]
    prec_temp = city_weather.groupby(['year', 'month'], as_index=False)['precipitation'].mean()
    prec_temp['temperature'] = city_weather.groupby(['year', 'month'], 
                                                    as_index=False)['avg_temperature'].mean()['avg_temperature']
      
    # create chart
    scatter = (Chart(prec_temp, width=300, height=200).mark_circle().encode(x="temperature", y="precipitation", \
        color="month:N")).configure_range(category={'scheme': 'viridis'})\
        .properties(title=cities.iloc[i]['name'] + ' (2011-2020)')
    vega = folium.features.VegaLite(scatter, width="100%", height="100%")
        
    # add chart to a marker     
    marker = folium.Marker([cities.iloc[i]['coordinates'].y, cities.iloc[i]['coordinates'].x], 
                         tooltip=cities.iloc[i]['name'])
    popup = folium.Popup()
    vega.add_to(popup)
    popup.add_to(marker)
    marker.add_to(marker_cluster)
     
feature_group = folium.FeatureGroup(name="Cities")
marker_cluster.add_to(feature_group)

# add minimap
minimap = plugins.MiniMap(toggle_display=True).add_to(m)
m.add_child(minimap)

# add mouse position
plugins.MousePosition(position="bottomleft").add_to(m)

# add layer options
folium.raster_layers.TileLayer("Stamen Terrain", name='Terrain').add_to(m)
folium.raster_layers.TileLayer("openstreetmap", name='Street Map').add_to(m)
folium.raster_layers.TileLayer("Stamen Toner", name='Toner').add_to(m)
folium.raster_layers.TileLayer("Cartodb Positron", name='Positron').add_to(m)
folium.raster_layers.TileLayer("Cartodb dark_matter", name='Dark matter').add_to(m)
feature_group.add_to(m)
folium.LayerControl().add_to(m)

m

Finally, we save the map as a standalone HTML page.

In [88]:
m.save(outfile= "map.html")